## Setup imports

In [46]:
import sys
import os
import logging
import datetime
import numpy as np
import torch
import monai.networks.nets as nets
from monai.transforms import (
    Compose,
    LoadImaged,
    AddChanneld,
    CropForegroundd,
    ToTensord,
    RandFlipd,
    RandAffined,
    SpatialPadd,
    Activationsd,
    Resized,
    AsDiscreted,
    AsDiscrete,
    GaussianSmoothd,
    SpatialCropd,
    MeanEnsembled,
    Activations,
)
from transforms import (
    CTWindowd,
    CTSegmentation,
    RelativeCropZd,
    RandGaussianNoised,
)
from monai.data import DataLoader, Dataset, PersistentDataset, CacheDataset
from torchsampler import ImbalancedDatasetSampler
from monai.transforms.croppad.batch import PadListDataCollate
from monai.utils import NumpyPadMode, set_determinism
from monai.utils.enums import Method
from monai.config import print_config
from monai.engines import EnsembleEvaluator
from monai.handlers import ROCAUC, StatsHandler
from ignite.metrics import Accuracy
from sklearn.model_selection import train_test_split
from trainer import Trainer
from validator import Validator
from tester import Tester
from utils import (
    multi_slice_viewer,
    setup_directories,
    get_data_from_info,
    large_image_splitter,
    calculate_class_imbalance,
    create_device,
    balance_training_data,
    balance_training_data2,
    transform_and_copy,
    convert_labels,
    load_mixed_images,
)
from test_data_loader import TestDataset
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

MONAI version: 0.5.dev2115
Numpy version: 1.20.2
Pytorch version: 1.8.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: c9990b040832792aa897b27a07180d4629cb8de3

Optional dependencies:
Pytorch Ignite version: 0.4.4
Nibabel version: 3.2.1
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 8.1.2
Tensorboard version: 2.4.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.9.0+cu111
ITK version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.59.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.8.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Setup directories

In [14]:
dirs = setup_directories()

## Setup torch device

In [15]:
# pass "cuda" to use the GPU
device, using_gpu = create_device("cuda")

## Load and randomize images

In [16]:
# HACKATON image and segmentation data
hackathon_dir = os.path.join(dirs["data"], 'HACKATHON')
map_fn = lambda x: (x[0], int(x[1]))
with open(os.path.join(hackathon_dir, "train.txt"), 'r') as fp:
    train_info_hackathon = [map_fn(entry.strip().split(',')) for entry in fp.readlines()]
image_dir = os.path.join(hackathon_dir, 'images', 'train')
seg_dir = os.path.join(hackathon_dir, 'segmentations', 'train')
_train_data_hackathon = get_data_from_info(image_dir, seg_dir, train_info_hackathon)
_train_data_hackathon = large_image_splitter(_train_data_hackathon, dirs["cache"], 4, only_label_one=True)
mixed_images = load_mixed_images(dirs["data"])
_train_data_hackathon.extend(mixed_images)
#copy_list = transform_and_copy(_train_data_hackathon, dirs['cache'])
#balance_training_data2(_train_data_hackathon, copy_list, seed=72)

# One channel
convert_labels(_train_data_hackathon, dtype=np.float32, as_array=True)
# Two channel
#convert_labels(_train_data_hackathon, dtype=np.int64, as_array=False)
    
# PSUF data
"""psuf_dir = os.path.join(dirs["data"], 'psuf')
with open(os.path.join(psuf_dir, "train.txt"), 'r') as fp:
    train_info = [entry.strip().split(',') for entry in fp.readlines()]
image_dir = os.path.join(psuf_dir, 'images')
train_data_psuf = get_data_from_info(image_dir, None, train_info)"""
# Split data into train, validate and test
train_split, test_data_hackathon = train_test_split(_train_data_hackathon, test_size=0.2, shuffle=True)#, random_state=42)
#copy_list = transform_and_copy(train_split, dirs['cache'])
#balance_training_data2(train_split, copy_list, seed=72)
train_data_hackathon, valid_data_hackathon = train_test_split(train_split, test_size=0.2, shuffle=True)#, random_state=43)

#convert_labels(train_data_hackathon, dtype=np.int64, as_array=False)
#convert_labels(valid_data_hackathon, dtype=np.int64, as_array=False)
#convert_labels(test_data_hackathon, dtype=np.int64, as_array=False)

#balance_training_data(train_data_hackathon, seed=72)
#balance_training_data(valid_data_hackathon, seed=73)
#balance_training_data(test_data_hackathon, seed=74)

Splitting large images...
original data len: 355
new data len: 396


## Setup transforms

In [17]:
# Crop foreground
crop_foreground = CropForegroundd(
    keys=["image"],
    source_key="image",
    margin=(5, 5, 0),
    select_fn = lambda x: x != 0
)
# Crop Z
crop_z = RelativeCropZd(keys=["image"], relative_z_roi=(0.05, 0.15))

# Window width and level (window center)
WW, WL = 1500, -600
ct_window = CTWindowd(keys=["image"], width=WW, level=WL)
# Random flip axis
rand_x_flip = RandFlipd(keys=["image"], spatial_axis=0, prob=0.50)
rand_y_flip = RandFlipd(keys=["image"], spatial_axis=1, prob=0.50)
rand_z_flip = RandFlipd(keys=["image"], spatial_axis=2, prob=0.50)
# Rand affine transform
rand_affine = RandAffined(
    keys=["image"],
    prob=0.50,
    rotate_range=(0, 0, np.pi/12),
    shear_range=(0.07, 0.07, 0.0),
    translate_range=(0, 0, 0),
    scale_range=(0.07, 0.07, 0.0),
    padding_mode="zeros"
)
# Pad image to have hight at least 30
spatial_pad = SpatialPadd(keys=["image"], spatial_size=(-1, -1, 30))
# Resize image x and y
resize_fator = 0.50
xy_size = int(512*resize_fator)
#resize = Resized(keys=["image"], spatial_size=(int(512*resize_fator), int(512*resize_fator), -1), mode="trilinear")
resize1 = Resized(keys=["image"], spatial_size=(-1, -1, 40), mode="area")
resize2 = Resized(keys=["image"], spatial_size=(xy_size, xy_size, -1), mode="area")
# spatioal crop
crop = SpatialCropd(keys=["image"], roi_start=(0, 0, 4), roi_end=(xy_size, xy_size, 36))
# Apply Gaussian noise
rand_gaussian_noise = RandGaussianNoised(keys=["image"], prob=0.25, mean=0.0, std=0.05)
# gaussian smooth
gaussian_noise_smooth = GaussianSmoothd(keys=["image"], sigma=(0.2, 0.2, 0.0))

#### Create transforms

In [18]:
common_transform = Compose([
    LoadImaged(keys=["image"]),
    ct_window,
    CTSegmentation(keys=["image"]),
    AddChanneld(keys=["image"]),
    crop_foreground,
    #crop_z,
    gaussian_noise_smooth,
    resize1,
    resize2,
    crop,
])
hackathon_train_transform = Compose([
    common_transform,
    rand_x_flip,
    rand_y_flip,
    rand_z_flip,
    rand_affine,
    rand_gaussian_noise,
    ToTensord(keys=["image"]),
]).flatten()
hackathon_valid_transfrom = Compose([
    common_transform,
    ToTensord(keys=["image"]),
]).flatten()
hackathon_test_transfrom = Compose([
    common_transform,
    ToTensord(keys=["image"]),
]).flatten()
psuf_transforms = Compose([
    LoadImaged(keys=["image"]),
    AddChanneld(keys=["image"]),
    ToTensord(keys=["image"]),
])

## Number of models

In [19]:
num_models = 5

## Split data

In [20]:
train_data = []
valid_data = []
tl = len(train_data_hackathon)
vl = int(tl * 0.2)
for i in range(num_models):
    train_data.append(train_data_hackathon[:(vl*i)] + train_data_hackathon[(vl*(i+1)):tl])
    valid_data.append(train_data_hackathon[(vl*i): (vl*(i+1))])

## Setup data

In [61]:
#set_determinism(seed=100)
train_datasets = [PersistentDataset(data=train_data[i], transform=hackathon_train_transform, cache_dir=dirs["persistent"]+f"_train_{i}") for i in range(num_models)]
valid_datasets = [PersistentDataset(data=valid_data[i], transform=hackathon_valid_transfrom, cache_dir=dirs["persistent"]+f"_valid_{i}") for i in range(num_models)]
test_dataset = PersistentDataset(data=test_data_hackathon[:], transform=hackathon_test_transfrom, cache_dir=dirs["persistent"])
_, n, p = calculate_class_imbalance(train_data_hackathon)
train_loaders = [DataLoader(
    train_datasets[i],
    batch_size=2,
    shuffle=True,
    pin_memory=using_gpu,
    num_workers=2,
    #sampler=ImbalancedDatasetSampler(train_data_hackathon, num_samples=2*p, callback_get_label=lambda x, i: x[i]['_label']),
    collate_fn=PadListDataCollate(Method.SYMMETRIC, NumpyPadMode.CONSTANT)
) for i in range(num_models)]
_, n, p = calculate_class_imbalance(valid_data_hackathon)
valid_loaders = [DataLoader(
    valid_datasets[i],
    batch_size=2,
    pin_memory=using_gpu,
    num_workers=2,
    #sampler=ImbalancedDatasetSampler(valid_data_hackathon, num_samples=2*p, callback_get_label=lambda x, i: x[i]['_label']),
    collate_fn=PadListDataCollate(Method.SYMMETRIC, NumpyPadMode.CONSTANT)
) for i in range(num_models)]
test_loader = DataLoader(
    test_dataset,
    batch_size=2,
    pin_memory=using_gpu,
    num_workers=2,
    collate_fn=PadListDataCollate(Method.SYMMETRIC, NumpyPadMode.CONSTANT)
)

## Create network, loss function, optimizer and scheduler

In [22]:
out_channels = 1
def create_model():
    #network = nets.EfficientNetBN("efficientnet-b4", spatial_dims=3, in_channels=1, num_classes=out_channels).to(device)
    network = nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=out_channels).to(device)
    # pos_weight for class imbalance
    _, n, p = calculate_class_imbalance(train_data_hackathon)
    print(n, p)
    ##############################################################
    # One channel
    pos_weight = torch.Tensor([n/p]).to(device)
    loss_function = torch.nn.BCEWithLogitsLoss(pos_weight)
    # Two channel
    #pos_weight = torch.Tensor([n, p]).to(device)
    #loss_function = torch.nn.CrossEntropyLoss(pos_weight)
    ##############################################################
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-3, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95, last_epoch=-1)
    return {'net': network, 'loss_fn': loss_function, 'opt': optimizer, 'lr_sch': scheduler}

models = [create_model() for i in range(num_models)]

220 58
220 58
220 58
220 58
220 58


## Setup validator and trainer

In [23]:
valid_post_transforms = Compose([
    Activationsd(keys="pred", sigmoid=True),
    #Activationsd(keys="pred", softmax=True),
])
def train(i, time):
    model = models[i]
    # Setup validator and trainer
    validator = Validator(
        device=device,
        val_data_loader=valid_loaders[i],
        network=model['net'],
        loss_function=model['loss_fn'],
        post_transform=valid_post_transforms,
        n_classes=out_channels,
        patience=30,
        amp=using_gpu,
        non_blocking=using_gpu
    )

    trainer = Trainer(
        device=device,
        out_dir=dirs["out"],
        out_name=f"DenseNet121_ensamble_{i}", # {i} je pomembna!!
        max_epochs=60,
        validation_epoch = 1,
        validation_interval = 1,
        train_data_loader=train_loaders[i],
        network=model['net'],
        optimizer=model['opt'],
        loss_function=model['loss_fn'],
        lr_scheduler=model['lr_sch'],
        validator=validator,
        amp=using_gpu,
        non_blocking=using_gpu
    )
    validator.early_stop_handler.set_trainer(trainer)
    trainer.run(time)
    return model['net']

## Run trainer

In [24]:
now = datetime.datetime.now()
trained_networks = [train(i, now) for i in range(num_models)]

Training started: 17/04/2021 17:13:39
INFO:ignite.engine.engine.Trainer:Engine run resuming from iteration 0, epoch 0 until 60 epochs
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 1/112 -- loss: 2.5760 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 2/112 -- loss: 2.5503 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 3/112 -- loss: 2.5843 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 4/112 -- loss: 5.0818 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 5/112 -- loss: 1.5398 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 6/112 -- loss: 1.5165 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 7/112 -- loss: 1.1266 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 8/112 -- loss: 2.7135 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 9/112 -- loss: 4.3381 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 10/112 -- loss: 0.8517 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 11/112 -- loss: 0.7947 
INFO:ignite.engine.engine.Tr

/opt/conda/lib/python3.7/site-packages/monai/handlers/stats_handler.py:196: UserWarning: ignoring non-scalar output in StatsHandler, make sure `output_transform(engine.state.output)` returns a scalar or dictionary of key and scalar pairs to avoid this warning. image:<class 'torch.Tensor'>
  " {}:{}".format(name, type(value))
/opt/conda/lib/python3.7/site-packages/monai/handlers/stats_handler.py:196: UserWarning: ignoring non-scalar output in StatsHandler, make sure `output_transform(engine.state.output)` returns a scalar or dictionary of key and scalar pairs to avoid this warning. label:<class 'torch.Tensor'>
  " {}:{}".format(name, type(value))
/opt/conda/lib/python3.7/site-packages/monai/handlers/stats_handler.py:196: UserWarning: ignoring non-scalar output in StatsHandler, make sure `output_transform(engine.state.output)` returns a scalar or dictionary of key and scalar pairs to avoid this warning. pred:<class 'torch.Tensor'>
  " {}:{}".format(name, type(value))
/opt/conda/lib/pytho

INFO:ignite.engine.engine.Validator:Got new best metric of Valid_AUC: 0.82
INFO:ignite.engine.engine.Validator:Epoch[1] Metrics -- Valid_ACC: 0.6364 Valid_AUC: 0.8200 Valid_Loss: 3.1065 
INFO:ignite.engine.engine.Validator:Key metric: Valid_AUC best value: 0.82 at epoch: 1
INFO:ignite.engine.engine.Validator:Epoch[1] Complete. Time taken: 00:00:51
INFO:ignite.engine.engine.Validator:Engine run complete. Time taken: 00:00:51
INFO:ignite.engine.engine.Trainer:Saved checkpoint at epoch: 1
INFO:ignite.engine.engine.Trainer:Epoch[1] Complete. Time taken: 00:04:15
INFO:ignite.engine.engine.Trainer:Epoch: 2/60, Iter: 1/112 -- loss: 1.6847 
INFO:ignite.engine.engine.Trainer:Epoch: 2/60, Iter: 2/112 -- loss: 1.6397 
INFO:ignite.engine.engine.Trainer:Epoch: 2/60, Iter: 3/112 -- loss: 1.4853 
INFO:ignite.engine.engine.Trainer:Epoch: 2/60, Iter: 4/112 -- loss: 1.1779 
INFO:ignite.engine.engine.Trainer:Epoch: 2/60, Iter: 5/112 -- loss: 1.4236 
INFO:ignite.engine.engine.Trainer:Epoch: 2/60, Iter: 6/

## Setup ensemble

In [105]:
mean_post_transforms = Compose([
    MeanEnsembled(
        keys=["pred0", "pred1", "pred2", "pred3", "pred4"],
        output_key="pred",
        # in this particular example, we use validation metrics as weights
        weights=[1, 1, 1, 1, 1],
    ),
    Activationsd(keys="pred", sigmoid=True),
])
val_handlers = [
    StatsHandler(output_transform=lambda x: None),
]
if out_channels > 1:
    to_onehot = AsDiscrete(to_onehot=True, n_classes=2)
else:
    to_onehot = lambda x: x
tester = EnsembleEvaluator(
    device=device,
    val_data_loader=test_loader,
    pred_keys=["pred0", "pred1", "pred2", "pred3", "pred4"],
    networks=trained_networks,
    post_transform=mean_post_transforms,
    key_val_metric={"Test_AUC": ROCAUC(average="micro", output_transform=lambda x: (x["pred"], to_onehot(x["label"])))},
    additional_metrics={"Test_ACC": Accuracy(output_transform=lambda x: (AsDiscrete(threshold_values=True)(x["pred"]), to_onehot(x["label"])))},
    val_handlers=val_handlers,
    amp=using_gpu,
    non_blocking=using_gpu,
)

## Run tester

In [106]:
tester.run()

INFO:ignite.engine.engine.EnsembleEvaluator:Engine run resuming from iteration 0, epoch 0 until 1 epochs
INFO:ignite.engine.engine.EnsembleEvaluator:Got new best metric of Test_AUC: 0.9595719381688466
INFO:ignite.engine.engine.EnsembleEvaluator:Epoch[1] Metrics -- Test_ACC: 0.8851 Test_AUC: 0.9596 
INFO:ignite.engine.engine.EnsembleEvaluator:Key metric: Test_AUC best value: 0.9595719381688466 at epoch: 1
INFO:ignite.engine.engine.EnsembleEvaluator:Epoch[1] Complete. Time taken: 00:00:23
INFO:ignite.engine.engine.EnsembleEvaluator:Engine run complete. Time taken: 00:00:23


## Setup tester

In [ ]:
"""tester = Tester(
    device=device,
    test_data_loader=test_loader,
    load_dir=train_output,
    out_dir=dirs["out"],
    network=network,
    n_classes=out_channels,
    post_transform=valid_post_transforms,
    non_blocking=using_gpu,
    amp=using_gpu
)"""

## Run tester

In [ ]:
#tester.run()

In [102]:
act = Activations(sigmoid=True) # One channel
#act = Activations(softmax=True)  # Two channel
#d = AsDiscrete(threshold_values=True)
test_outputs_global = [[] for i in range(num_models)]
test_labels_global = [[] for i in range(num_models)]
with torch.no_grad():
    for test_data in test_loader:
        for i in range(num_models):
            trained_networks[i].eval()
            test_images, test_labels = test_data["image"].to(device), test_data["label"].to(device)
            test_outputs = act(trained_networks[i](test_images))
            test_outputs = test_outputs.detach().cpu().numpy().ravel()
            test_labels = test_labels.detach().cpu().numpy().ravel()
            test_outputs_global[i].extend(test_outputs.tolist())
            test_labels_global[i].extend(test_labels.tolist())

test_outputs_global = np.transpose(np.array(test_outputs_global))
test_labels_global = np.transpose(np.array(test_labels_global))
test_outputs_mean = np.empty(0)
test_outputs_d = np.empty(0)
d = lambda x: 1 if (x > 0.5) else 0
for a in test_outputs_global:
    m = np.mean(a)
    test_outputs_mean = np.append(test_outputs_mean, m)
    test_outputs_d = np.append(test_outputs_d, d(m))
for i in range(len(test_outputs_mean)):
    print(test_outputs_mean[i], test_outputs_d[i], test_labels_global[i][0])

0.058888216875493525 0.0 0.0
0.27199710384011266 0.0 0.0
0.010197634631185792 0.0 0.0
0.0997410211712122 0.0 0.0
0.039958341419696806 0.0 0.0
0.04369748141616583 0.0 0.0
0.06884096041321755 0.0 0.0
0.036350092850625515 0.0 0.0
0.07406269162893295 0.0 0.0
0.6812442660331726 1.0 1.0
0.37463057935237887 0.0 1.0
0.05116980634629727 0.0 0.0
0.4722948491573334 0.0 1.0
0.6519052147865295 1.0 1.0
0.017926387721672654 0.0 0.0
0.08852747678756714 0.0 0.0
0.34081542640924456 0.0 0.0
0.025823887810111044 0.0 0.0
0.9478948235511779 1.0 1.0
0.6934697270393372 1.0 1.0
0.2450999293476343 0.0 0.0
0.16308507397770883 0.0 0.0
0.14123211428523064 0.0 0.0
0.7054976105690003 1.0 1.0
0.07043303288519383 0.0 1.0
0.025089913280680776 0.0 0.0
0.16416353583335877 0.0 1.0
0.022521707011037506 0.0 0.0
0.9012171506881714 1.0 1.0
0.6769929826259613 1.0 1.0
0.05523233981803059 0.0 0.0
0.02219142117537558 0.0 0.0
0.026362506579607724 0.0 0.0
0.6065349966287613 1.0 1.0
0.11728044897317887 0.0 0.0
0.3952154964208603 0.0

## Plot results

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

#### Loss

In [ ]:
"""rain_loss = np.hsplit(np.loadtxt(os.path.join(train_output, 'log_loss.txt')), 2)
valid_loss = np.hsplit(np.loadtxt(os.path.join(train_output, 'log_Valid_Loss.txt')), 2)"""

In [ ]:
"""fig, ax = plt.subplots()
ax.plot(train_loss[0], train_loss[1], valid_loss[0], valid_loss[1])
ax.set(xlabel='interation', ylabel='loss',
       title='Loss')
ax.grid()
plt.show()"""

#### AUC and ACC

In [ ]:
"""valid_auc = np.hsplit(np.loadtxt(os.path.join(train_output, 'log_Valid_AUC.txt')), 2)
valid_acc = np.hsplit(np.loadtxt(os.path.join(train_output, 'log_Valid_ACC.txt')), 2)"""

In [ ]:
"""fig, ax = plt.subplots()
ax.plot(valid_auc[0], valid_auc[1], valid_acc[0], valid_acc[1])
ax.set(xlabel='interation', ylabel='AUC and ACC',
       title='AUC and ACC')
ax.grid()
plt.show()"""